### Setup

##### Install

In [67]:
%pip install twilio python-dotenv matplotlib numpy diagrams graphviz mermaid-py --quiet

Note: you may need to restart the kernel to use updated packages.


##### Init Client

##### Functions

In [70]:
import os
from twilio.rest import Client
from matplotlib.patches import FancyBboxPatch, ConnectionPatch
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches

import IPython

from mermaid import Mermaid
from IPython.display import display

# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure
account_sid = os.environ["TWILIO_ACCOUNT_SID"]
auth_token = os.environ["TWILIO_AUTH_TOKEN"]
client = Client(account_sid, auth_token)


In [141]:
from IPython.display import display, HTML
import xml.dom.minidom

def pretty_print_twiml(twiml_response):
    """
    Takes a TwiML VoiceResponse object and renders it in a Twilio-branded pretty HTML box.
    
    Args:
        twiml_response: A twilio.twiml.voice_response.VoiceResponse object
    
    Returns:
        str: Pretty formatted XML string (also displayed in a styled box)
    """
    # Convert TwiML response to pretty XML
    xml_string = str(twiml_response)
    pretty_xml = xml.dom.minidom.parseString(xml_string).toprettyxml(indent="  ")
    pretty_xml = '\n'.join(pretty_xml.split('\n')[1:])  # Remove XML declaration

    # Escape HTML special characters
    escaped_xml = pretty_xml.replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")

    # Twilio-style display box
    style = """
    <style>
    .twilio-xml-box {
        background: #0B0D17;
        border: 1px solid #661847;
        padding: 20px;
        border-radius: 12px;
        font-family: 'Courier New', monospace;
        font-size: 0.95em;
        color: #FFFFFF;
        white-space: pre-wrap;
        line-height: 1.5;
        box-shadow: 0 0 12px rgba(242, 47, 70, 0.4);
        margin: 20px auto;
        width: 90%;
    }
    .twilio-xml-box span.tag {
        color: #F22F46;
    }
    .twilio-xml-box span.attr {
        color: #0DC5FB;
    }
    </style>
    """

    html_output = f"""
    {style}
    <div class="twilio-xml-box">{escaped_xml}</div>
    """

    display(HTML(html_output))
    return pretty_xml.strip()


In [134]:
import time
from IPython.display import display, clear_output, HTML

def animate_call_flow_html(sid=None, delay=1.5):
    steps = [
        "Connecting Call",
        "AI Agent connected",
        "Text to Speech initiated",
        "Call Ended"
    ]

    style = """
    <style>
    @keyframes gradientFlow {
        0% { background-position: 0% 50%; }
        50% { background-position: 100% 50%; }
        100% { background-position: 0% 50%; }
    }
    .twilio-flow {
        font-family: 'Segoe UI', sans-serif;
        font-size: 1.6em;
        color: white;
        font-weight: 600;
        padding: 18px 24px;
        border-radius: 12px;
        text-align: center;
        background: linear-gradient(-45deg, #0DC5FB, #661847, #4B0082, #F22F46);
        background-size: 400% 400%;
        animation: gradientFlow 6s ease infinite;
        box-shadow: 0 0 20px rgba(0,0,0,0.3);
        margin: 20px auto;
        width: 70%;
    }
    .twilio-sid {
        font-size: 0.95em;
        margin-top: 10px;
        color: #888;
    }
    </style>
    """

    flow = ""
    for step in steps:
        flow += f"{step} ➝ "
        clear_output(wait=True)
        html = f"""
        {style}
        <div class="twilio-flow">{flow.strip(' ➝ ')}</div>
        """
        display(HTML(html))
        time.sleep(delay)

    if sid:
        final = f"""
        {style}
        <div class="twilio-flow">Call Successful!</div>
        <div class="twilio-sid">Call SID: <code>{sid}</code></div>
        """
        clear_output(wait=True)
        display(HTML(final))


___

___



# Twilio Brasil - Signal São Paulo 2025



___

In [147]:
import time
from IPython.display import display, clear_output, HTML

# Messages to cycle through
messages = [
    "Welcome, Builders!",
    "Bienvenidos, Creadores!",
    "Bem-vindos, Criadores!",
    "Bienvenidas, Creadoras!",
    "Bem-vindas, Criadoras!"
]

# Final message
final_message = "TWILIO SIGNAL SÃO PAULO 2025"

# Shared CSS for animated gradient box
style = """
<style>
@keyframes gradient {
    0% { background-position: 0% 50%; }
    50% { background-position: 100% 50%; }
    100% { background-position: 0% 50%; }
}
.welcome-box {
    font-family: 'Segoe UI', sans-serif;
    font-size: 2em;
    font-weight: bold;
    color: white;
    padding: 30px;
    text-align: center;
    border-radius: 16px;
    background: linear-gradient(-45deg, #0DC5FB, #661847, #4B0082, #F22F46);
    background-size: 400% 400%;
    animation: gradient 8s ease infinite;
    box-shadow: 0 0 20px rgba(0,0,0,0.4);
    margin: 40px auto;
    width: 70%;
}
</style>
"""

# Animation loop
for _ in range(2):  # Number of cycles
    for msg in messages:
        clear_output(wait=True)
        html = f"{style}<div class='welcome-box'>{msg}</div>"
        display(HTML(html))
        time.sleep(1.5)

# Final SIGNAL banner
clear_output(wait=True)
html = f"{style}<div class='welcome-box'>{final_message}</div>"
display(HTML(html))


___

## Architectural Overview

In [114]:
from IPython.display import display
from mermaid import Mermaid

mermaid_string = """
%%{init: {
  "theme": "base",
  "themeVariables": {
    "background": "#0B0D17",
    "primaryColor": "#12284C",
    "primaryBorderColor": "#661847",
    "primaryTextColor": "#FFFFFF",
    "tertiaryColor": "#0DC5FB",
    "tertiaryBorderColor": "#0B3D91",
    "tertiaryTextColor": "#FFFFFF",
    "noteBkgColor": "#4B0082",
    "noteTextColor": "#FFFFFF",
    "clusterBkg": "#0B0D17",
    "clusterBorder": "#1D1F2B"
  },
  "flowchart": {
    "useMaxWidth": true,
    "htmlLabels": true,
    "wrap": true
  }
}}%%

graph TD

    %% Step Labels - placed left of clusters
    Step1Label[/"Step 1: Inbound / Outbound Call"/]:::steplabel
    Step2Label[/"Step 2: Twilio Platform"/]:::steplabel
    Step3Label[/"Step 3: External AI Service"/]:::steplabel

    Step1Label --> PSTN
    Step2Label --> VoiceAPI
    Step3Label --> ExtService

    %% Step 1: Channels
    subgraph A[" "]
        PSTN["PSTN"]
        SIP["SIP"]
        WhatsApp["WhatsApp"]
        WebRTC["WebRTC / In-App"]
    end

    %% Step 2: Twilio Core
    subgraph B[" "]
        VoiceAPI["Voice API"]
        subgraph Orchestration["Orchestration Layer"]
            ConvRelay["Conversation Relay"]
            Studio["Studio"]
            TwiML["TwiML Bins"]
            Functions["Functions"]
            Webhooks["External Webhooks"]
        end
    end

    %% Step 3: External AI
    subgraph C[" "]
        ExtService["Relay Backend"]
        subgraph LLMs["LLM Models"]
            GPT["GPT"]
            Gemini["Gemini"]
            Claude["Claude"]
            DeepSeek["DeepSeek"]
        end
    end

    %% Connections
    PSTN --> VoiceAPI
    SIP --> VoiceAPI
    WhatsApp --> VoiceAPI
    WebRTC --> VoiceAPI

    VoiceAPI --> Studio
    VoiceAPI --> TwiML
    VoiceAPI --> Functions
    VoiceAPI --> Webhooks
    VoiceAPI --> ConvRelay

    ConvRelay -.->|"STT + Events"| ExtService
    ExtService -.->|"TTS Response"| ConvRelay

    ExtService --> GPT
    ExtService --> Gemini
    ExtService --> Claude
    ExtService --> DeepSeek

    %% Styling
    classDef entrypoint fill:#0DC5FB,stroke:#0B3D91,stroke-width:2px,color:#ffffff
    classDef twilio fill:#12284C,stroke:#661847,stroke-width:2px,color:#ffffff
    classDef relay fill:#F22F46,stroke:#0B3D91,stroke-width:2px,color:#ffffff
    classDef backend fill:#4B0082,stroke:#F22F46,stroke-width:2px,color:#ffffff
    classDef llm fill:#ffffff,stroke:#4B0082,stroke-width:2px,color:#4B0082
    classDef steplabel fill:#1D1F2B,stroke:#333333,stroke-width:1px,color:#ffffff

    class PSTN,SIP,WhatsApp,WebRTC entrypoint
    class VoiceAPI,Studio,TwiML,Functions,Webhooks twilio
    class ConvRelay relay
    class ExtService backend
    class GPT,Gemini,Claude,DeepSeek llm
"""

display(Mermaid(mermaid_string))


___

## Twilio Voice API

**Realizar uma chamada**

In [ ]:
call = client.calls.create(
    from_="+551150397615",
    to="+5511968432422",
    twiml="<Response> <Say> Ahoy! Signal São Paulo! </Say> </Response>",
)

animate_call_flow_html(call.sid)


## TwiML 101

**Controle de Chamadas com a Twilio Markup Language**

#### TwiML `<Say>` 
Text to Speech (TTS)

In [ ]:
# SAY

<?xml version="1.0" encoding="UTF-8"?>
<Response>
  <Say language="fr-FR">Bonjour!</Say>
</Response>

#### TwiML `<Gather>` 

** Real-time transcription (speech to text  or STT) and digits input (DTMF)

In [ ]:
# GATHER

<?xml version="1.0" encoding="UTF-8"?>
<Response>
  <Gather input="speech dtmf" finishOnKey="#" timeout="5">
    <Say>
      Please say something or press * to access the main menu
    </Say>
  </Gather>
  <Say>We didn't receive any input. Goodbye!</Say>
</Response>

#### Usando TwiML em Python com a lib da Twilio 

In [ ]:
## SAY usando a biblioteca Twilio

from twilio.twiml.voice_response import VoiceResponse

response = VoiceResponse()

response.say(
    language="fr-FR", 
    text="Bonjour!"
    )

pretty_print_twiml(response)

In [ ]:
# GATHER usando a biblioteca Twilio

from twilio.twiml.voice_response import Gather, VoiceResponse, Say

response = VoiceResponse()
gather = Gather(action='/process_gather.php', method='GET')

gather.say('Please enter your account number,\nfollowed by the pound sign')
response.append(gather)

response.say('We didn\'t receive any input. Goodbye!')

import xml.dom.minidom

pretty_print_twiml(response)

### Realizando uma chamada com TwiML Say e Gather

In [ ]:
# Fazendo uma chamada usando SAY e GATHER
from twilio.twiml.voice_response import VoiceResponse, Gather

response = VoiceResponse()

# Gather DTMF or speech
gather = Gather(input="speech dtmf", timeout=5, action="/handle_input", method="POST") #TO DO: Add route to handle input
gather.say("Por favor, me fale como posso ajudar você hoje. Ou digite um número qualquer seguido de cerquilha.")
response.append(gather)

# Fallback if no input
response.say("Poxa! Não recebemos nenhuma resposta. Tchau tchau!")

pretty_print_twiml(str(response))  # optional: view generated XML

# Make the call
call = client.calls.create(
    from_="+551150397615",
    to="+5511968432422",
    twiml=str(response)
)

print(call.sid)


___

## Twilio Conversation Relay

### Conectando chamadas com o TwiML `<Connect>`

In [ ]:
<?xml version="1.0" encoding="UTF-8"?>
<Response>
  <Connect action="https://myhttpserver.com/connect_action">
    <ConversationRelay url="wss://mywebsocketserver.com/websocket" welcomeGreeting="Hi! Ask me anything!" />
  </Connect>
</Response>

### Conversation Relay usando a biblioteca da Twilio para Python

In [ ]:
from twilio.twiml.voice_response import Connect, ConversationRelay, Language, VoiceResponse

response = VoiceResponse()
connect = Connect()

conversationrelay = ConversationRelay(
    url='wss://mywebsocketserver.com/websocket')

conversationrelay.language(
    code='pt-BR',
    tts_provider='ElevenLabs',
    voice='UgBBYS2sOqTuMpoF3BR0',
    transcription_provider='google',
    speech_model='telephony')

conversationrelay.language(
    code='es-US', tts_provider='google', voice='es-US-Chirp3-HD-Kore')

connect.append(conversationrelay)

response.append(connect)

print(response)

## LET'S BUILD

### Fazendo chamadas com `<ConversationRelay>`

In [ ]:
# Fazendo uma chamada usando Conversation Relay

from twilio.twiml.voice_response import VoiceResponse, Connect, ConversationRelay

# Get your ngrok domain from .env file
ngrok_domain = os.getenv('NGROK_DOMAIN', 'your-custom-domain.ngrok-free.app')

# Create TwiML with Conversation Relay
response = VoiceResponse()
connect = Connect()

# Create ConversationRelay pointing to your WebSocket server
conversation_relay = ConversationRelay(
    url=f'wss://{ngrok_domain}/websocket',
    welcome_greeting="Olá! Eu sou o assistente virtual da Signal São Paulo. Que evento lindo, hein? Como posso ajudar você hoje?",
    language='pt-BR',
    transcription_provider='Deepgram',
    speech_model='nova-2-general',
    tts_provider='ElevenLabs',
    voice='7u8qsX4HQsSHJ0f8xsQZ',
)

# Configure language settings for Brazilian Portuguese
conversation_relay.language(
    code='pt-BR',
    tts_provider='ElevenLabs',
    voice='7u8qsX4HQsSHJ0f8xsQZ',
    transcription_provider='Deepgram',
    speech_model='nova-2-general'
)

connect.append(conversation_relay)
response.append(connect)

# Make the call with Conversation Relay
call = client.calls.create(
    from_="+551150397615",
    to="+5511968432422",
    twiml=str(response)
)

print(f"Call SID: {call.sid}")
print(f"WebSocket URL: wss://{ngrok_domain}")
pretty_print_twiml(response)

Call SID: CA384d6e34b5337cd5e1743ee7ac89a176
WebSocket URL: wss://owlbank.ngrok.io
<Response>
  <Connect>
    <ConversationRelay language="pt-BR" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" url="wss://owlbank.ngrok.io/websocket" voice="7u8qsX4HQsSHJ0f8xsQZ" welcomeGreeting="Olá! Eu sou o assistente virtual da Signal São Paulo. Que evento lindo, hein? Como posso ajudar você hoje?">
      <Language code="pt-BR" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" voice="7u8qsX4HQsSHJ0f8xsQZ"/>
    </ConversationRelay>
  </Connect>
</Response>


'<Response>\n  <Connect>\n    <ConversationRelay language="pt-BR" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" url="wss://owlbank.ngrok.io/websocket" voice="7u8qsX4HQsSHJ0f8xsQZ" welcomeGreeting="Olá! Eu sou o assistente virtual da Signal São Paulo. Que evento lindo, hein? Como posso ajudar você hoje?">\n      <Language code="pt-BR" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" voice="7u8qsX4HQsSHJ0f8xsQZ"/>\n    </ConversationRelay>\n  </Connect>\n</Response>'

## `attributes` 

**Vamos melhorar isso um pouco?**

In [ ]:
# Aprimorando as chamadas feitas com Conversation Relay

from twilio.twiml.voice_response import VoiceResponse, Connect, ConversationRelay

# Get your ngrok domain from .env file
ngrok_domain = os.getenv('NGROK_DOMAIN', 'your-custom-domain.ngrok-free.app')

# Create TwiML with Conversation Relay
response = VoiceResponse()
connect = Connect()

# Create ConversationRelay pointing to your WebSocket server
conversation_relay = ConversationRelay(
    url=f'wss://{ngrok_domain}/websocket',
    welcome_greeting="Olá! Eu sou o assistente virtual da Signal São Paulo. Que evento lindo, hein? Como posso ajudar você hoje?",
    welcomeGreetingInterruptible=True,  # Allow user to interrupt the welcome greeting
    language='pt-BR',
    transcription_provider='Deepgram',
    speech_model='nova-2-general',
    tts_provider='ElevenLabs',
    voice='7u8qsX4HQsSHJ0f8xsQZ',
    interruptible=True,  # Allow user to interrupt the assistant
    dtmfDetection=True,  # Enable DTMF detection
    reportInputDuringAgentSpeech=True,  # Report input during agent speech
    preemptible=True,  # Allow the assistant to preempt the user
    hints='Twilio, Conversation Relay, Signal São Paulo, Owl Bank',
    debug='debugging, speaker-events, tokens-played', # debbugging options
    elevenlabsTextNormalization='on',
)

# Configure language settings for Brazilian Portuguese
conversation_relay.language(
    code='pt-BR',
    tts_provider='ElevenLabs',
    voice='7u8qsX4HQsSHJ0f8xsQZ',
    transcription_provider='Deepgram',
    speech_model='nova-2-general'
)

connect.append(conversation_relay)
response.append(connect)

# Make the call with Conversation Relay
call = client.calls.create(
    from_="+551150397615",
    to="+5511968432422",
    twiml=str(response)
)

print(f"Call SID: {call.sid}")
print(f"WebSocket URL: wss://{ngrok_domain}")
pretty_print_twiml(response)

Call SID: CAbbd5cfb081b2321b3fde7092e064435d
WebSocket URL: wss://owlbank.ngrok.io
<Response>
  <Connect>
    <ConversationRelay debug="debugging, speaker-events, tokens-played" dtmfDetection="true" elevenlabsTextNormalization="on" hints="Twilio, Conversation Relay, Signal São Paulo, Owl Bank" interruptible="true" language="pt-BR" preemptible="true" reportInputDuringAgentSpeech="true" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" url="wss://owlbank.ngrok.io/websocket" voice="7u8qsX4HQsSHJ0f8xsQZ" welcomeGreeting="Olá! Eu sou o assistente virtual da Signal São Paulo. Que evento lindo, hein? Como posso ajudar você hoje?" welcomeGreetingInterruptible="true">
      <Language code="pt-BR" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" voice="7u8qsX4HQsSHJ0f8xsQZ"/>
    </ConversationRelay>
  </Connect>
</Response>


'<Response>\n  <Connect>\n    <ConversationRelay debug="debugging, speaker-events, tokens-played" dtmfDetection="true" elevenlabsTextNormalization="on" hints="Twilio, Conversation Relay, Signal São Paulo, Owl Bank" interruptible="true" language="pt-BR" preemptible="true" reportInputDuringAgentSpeech="true" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" url="wss://owlbank.ngrok.io/websocket" voice="7u8qsX4HQsSHJ0f8xsQZ" welcomeGreeting="Olá! Eu sou o assistente virtual da Signal São Paulo. Que evento lindo, hein? Como posso ajudar você hoje?" welcomeGreetingInterruptible="true">\n      <Language code="pt-BR" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" voice="7u8qsX4HQsSHJ0f8xsQZ"/>\n    </ConversationRelay>\n  </Connect>\n</Response>'

## Como Escalar com Conversation Relay?

### `<Language>` 

**Multi-Idiomas Nativo, Flexibilidade de Modelos para TTS e STT**

In [ ]:
# Suporte nativo a múltiplos idiomas

from twilio.twiml.voice_response import VoiceResponse, Connect, ConversationRelay

# Get your ngrok domain from .env file
ngrok_domain = os.getenv('NGROK_DOMAIN', 'your-custom-domain.ngrok-free.app')

# Create TwiML with Conversation Relay
response = VoiceResponse()
connect = Connect()

# Create ConversationRelay pointing to your WebSocket server
conversation_relay = ConversationRelay(
    url=f'wss://{ngrok_domain}/websocket',
    welcome_greeting="Olá! Eu sou o assistente virtual do Sígnal São Paulo. Que evento lindo, hein? Como posso ajudar você hoje?",
    welcomeGreetingInterruptible=True,  # Allow user to interrupt the welcome greeting
    language='pt-BR',
    debug='debugging, speaker-events, tokens-played', # debbugging options
    interruptible=True,  # Allow user to interrupt the assistant
    dtmfDetection=True,  # Enable DTMF detection
    reportInputDuringAgentSpeech=True,  # Report input during agent speech
    preemptible=True,  # Allow the assistant to preempt the user
    hints='Twilio, Conversation Relay, Signal São Paulo, Owl Bank',
    elevenlabsTextNormalization='on',
)

# Configure language settings for Brazilian Portuguese
conversation_relay.language(
    code='pt-BR',
    tts_provider='ElevenLabs',
    voice='7u8qsX4HQsSHJ0f8xsQZ',
    transcription_provider='Deepgram',
    speech_model='nova-2-general',
    customParameter='change_to_pt-BR'
)

# Configure language settings for US Spanish
conversation_relay.language(
    code='es-US',
    tts_provider='ElevenLabs',
    voice='7u8qsX4HQsSHJ0f8xsQZ',
    transcription_provider='Deepgram',
    speech_model='nova-2-general',
    customParameter='change_to_es-US'
)

# Configure language settings for US English
conversation_relay.language(
    code='en-US',
    tts_provider='ElevenLabs',
    voice='7u8qsX4HQsSHJ0f8xsQZ',
    transcription_provider='Deepgram',
    speech_model='nova-2-general',
    customParameter="change_to_en-US"
)

connect.append(conversation_relay)
response.append(connect)

# Make the call with Conversation Relay
call = client.calls.create(
    from_="+551150397615",
    to="+5511968432422",
    twiml=str(response)
)

print(f"Call SID: {call.sid}")
print(f"WebSocket URL: wss://{ngrok_domain}")
pretty_print_twiml(response)

Call SID: CA88d6eb89646ad979481871af63830b38
WebSocket URL: wss://owlbank.ngrok.io
<Response>
  <Connect>
    <ConversationRelay debug="debugging, speaker-events, tokens-played" dtmfDetection="true" elevenlabsTextNormalization="on" hints="Twilio, Conversation Relay, Signal São Paulo, Owl Bank" interruptible="true" language="pt-BR" preemptible="true" reportInputDuringAgentSpeech="true" url="wss://owlbank.ngrok.io/websocket" welcomeGreeting="Olá! Eu sou o assistente virtual do Sígnal São Paulo. Que evento lindo, hein? Como posso ajudar você hoje?" welcomeGreetingInterruptible="true">
      <Language code="pt-BR" customParameter="change_to_pt-BR" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" voice="7u8qsX4HQsSHJ0f8xsQZ"/>
      <Language code="es-US" customParameter="change_to_es-US" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" voice="7u8qsX4HQsSHJ0f8xsQZ"/>
      <Language code="en-US" customParameter="cha

'<Response>\n  <Connect>\n    <ConversationRelay debug="debugging, speaker-events, tokens-played" dtmfDetection="true" elevenlabsTextNormalization="on" hints="Twilio, Conversation Relay, Signal São Paulo, Owl Bank" interruptible="true" language="pt-BR" preemptible="true" reportInputDuringAgentSpeech="true" url="wss://owlbank.ngrok.io/websocket" welcomeGreeting="Olá! Eu sou o assistente virtual do Sígnal São Paulo. Que evento lindo, hein? Como posso ajudar você hoje?" welcomeGreetingInterruptible="true">\n      <Language code="pt-BR" customParameter="change_to_pt-BR" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" voice="7u8qsX4HQsSHJ0f8xsQZ"/>\n      <Language code="es-US" customParameter="change_to_es-US" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" voice="7u8qsX4HQsSHJ0f8xsQZ"/>\n      <Language code="en-US" customParameter="change_to_en-US" speechModel="nova-2-general" transcriptionProvider="Deepgram" t

### Engajamento de Clientes Multi-Agente

**Aprimorando a experiência com agentes especialistas**

### Observabiidade com Conversational Intelligence

In [ ]:
# Aprimorando as chamadas feitas com Conversation Relay

from twilio.twiml.voice_response import VoiceResponse, Connect, ConversationRelay

# Get your ngrok domain from .env file
ngrok_domain = os.getenv('NGROK_DOMAIN', 'your-custom-domain.ngrok-free.app')

# Create TwiML with Conversation Relay
response = VoiceResponse()
connect = Connect()

# Create ConversationRelay pointing to your WebSocket server
conversation_relay = ConversationRelay(
    url=f'wss://{ngrok_domain}/websocket',
    welcome_greeting="Olá! Eu sou o assistente virtual do Sígnal São Paulo. Que evento lindo, hein? Como posso ajudar você hoje?",
    welcomeGreetingInterruptible=True,  # Allow user to interrupt the welcome greeting
    language='pt-BR',
    transcription_provider='Deepgram',
    speech_model='nova-2-general',
    tts_provider='ElevenLabs',
    voice='7u8qsX4HQsSHJ0f8xsQZ',
    interruptible=True,  # Allow user to interrupt the assistant
    dtmfDetection=True,  # Enable DTMF detection
    reportInputDuringAgentSpeech=True,  # Report input during agent speech
    preemptible=True,  # Allow the assistant to preempt the user
    hints='Twilio, Conversation Relay, Signal São Paulo, Owl Bank',
    debug='debugging, speaker-events, tokens-played', # debbugging options
    elevenlabsTextNormalization='on',
    intelligenceService='GAde9c513fd3914897cac25df18f3203b7'
)

# Configure language settings for Brazilian Portuguese
conversation_relay.language(
    code='pt-BR',
    tts_provider='ElevenLabs',
    voice='7u8qsX4HQsSHJ0f8xsQZ',
    transcription_provider='Deepgram',
    speech_model='nova-2-general'
)

connect.append(conversation_relay)
response.append(connect)

# Make the call with Conversation Relay
call = client.calls.create(
    from_="+551150397615",
    to="+5511968432422",
    twiml=str(response)
)

print(f"Call SID: {call.sid}")
print(f"WebSocket URL: wss://{ngrok_domain}")
pretty_print_twiml(response)

Call SID: CA9071c985d461e9ea9f095e89d380e524
WebSocket URL: wss://owlbank.ngrok.io
<Response>
  <Connect>
    <ConversationRelay debug="debugging, speaker-events, tokens-played" dtmfDetection="true" elevenlabsTextNormalization="on" hints="Twilio, Conversation Relay, Signal São Paulo, Owl Bank" intelligenceService="GAde9c513fd3914897cac25df18f3203b7" interruptible="true" language="pt-BR" preemptible="true" reportInputDuringAgentSpeech="true" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" url="wss://owlbank.ngrok.io/websocket" voice="7u8qsX4HQsSHJ0f8xsQZ" welcomeGreeting="Olá! Eu sou o assistente virtual do Sígnal São Paulo. Que evento lindo, hein? Como posso ajudar você hoje?" welcomeGreetingInterruptible="true">
      <Language code="pt-BR" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" voice="7u8qsX4HQsSHJ0f8xsQZ"/>
    </ConversationRelay>
  </Connect>
</Response>


'<Response>\n  <Connect>\n    <ConversationRelay debug="debugging, speaker-events, tokens-played" dtmfDetection="true" elevenlabsTextNormalization="on" hints="Twilio, Conversation Relay, Signal São Paulo, Owl Bank" intelligenceService="GAde9c513fd3914897cac25df18f3203b7" interruptible="true" language="pt-BR" preemptible="true" reportInputDuringAgentSpeech="true" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" url="wss://owlbank.ngrok.io/websocket" voice="7u8qsX4HQsSHJ0f8xsQZ" welcomeGreeting="Olá! Eu sou o assistente virtual do Sígnal São Paulo. Que evento lindo, hein? Como posso ajudar você hoje?" welcomeGreetingInterruptible="true">\n      <Language code="pt-BR" speechModel="nova-2-general" transcriptionProvider="Deepgram" ttsProvider="ElevenLabs" voice="7u8qsX4HQsSHJ0f8xsQZ"/>\n    </ConversationRelay>\n  </Connect>\n</Response>'

### Verificando os resultados com Conversational Intelligence

In [ ]:
# List Transcripts Incrementally
from datetime import datetime

# Initialize or use existing last_update_date
if 'last_update_date' not in globals():
    # First run - get transcripts from last 24 hours to avoid missing recent ones
    from datetime import timedelta
    last_update_date = datetime.now() - timedelta(days=1)

last_update_time = last_update_date.strftime('%Y-%m-%dT%H:%M:%SZ')
print(f"Fetching transcripts after: {last_update_time}")

transcripts = client.intelligence.v2.transcripts.list(
    limit=100,
    after_date_created=last_update_time,
    after_start_time=last_update_time,
)

print(f"Found {len(transcripts)} new transcripts:\n")

for record in transcripts:
    for key, value in vars(record).items():
        print(f"{key}: {value}")
    print("\n---\n")

# Update the timestamp for next run
last_update_date = datetime.now()
print(f"Updated last_update_date to: {last_update_date}")

Fetching transcripts after: 2025-06-17T09:27:19Z
Found 1 new transcripts:

_version: <Twilio.Intelligence.V2>
account_sid: ACdf269d13302fe4635ab695fb1b477395
service_sid: GAde9c513fd3914897cac25df18f3203b7
sid: GT17c5c6b21b6e45d88549a5b148283baf
date_created: 2025-06-18 01:16:25+00:00
date_updated: 2025-06-18 01:16:36+00:00
status: completed
channel: {'media_properties': {'source': 'ConversationRelay', 'reference_sids': {'call_sid': 'CAfaa1d43461385c3356b60e34f4be82c8'}, 'source_sid': 'VXa4a49109680b7037acdfbd86926f9069', 'media_url': None}, 'participants': [{'user_id': None, 'channel_participant': 1, 'media_participant_id': 'Virtual Agent', 'image_url': None, 'full_name': 'Virtual Agent', 'role': 'Virtual Agent', 'email': None}, {'user_id': None, 'channel_participant': 2, 'media_participant_id': '+551150397615', 'image_url': None, 'full_name': '+551150397615', 'role': 'Customer', 'email': None}], 'type': 'voice'}
data_logging: False
language_code: pt-BR
customer_key: None
media_start_

In [ ]:
# List Operator Results for all Transcripts found incrementally
print(f"Getting operator results for {len(transcripts)} transcripts...\n")

for transcript_record in transcripts:
    print(f"TRANSCRIPT: {transcript_record.sid}")
    print("=" * 80)
    
    try:
        operator_results = client.intelligence.v2.transcripts(
            transcript_record.sid
        ).operator_results.list(limit=100)
        
        print(f"Found {len(operator_results)} operator results:\n")
        
        for i, record in enumerate(operator_results, 1):
            print(f"OPERATOR RESULT #{i}")
            print("=" * 60)
            print(f"Name: {record.name}")
            print(f"Type: {record.operator_type}")
            print(f"Operator SID: {record.operator_sid}")
            
            # Handle different result types
            if record.operator_type == 'text-generation' and record.text_generation_results:
                print(f"\nText Generation Result:")
                print("-" * 30)
                result_text = record.text_generation_results.get('result', 'No result available')
                print(f"{result_text}")
                
            elif record.operator_type == 'extract' and record.extract_results:
                print(f"\nExtraction Results:")
                print("-" * 30)
                for entity_type, entities in record.extract_results.items():
                    print(f"  {entity_type}: {entities}")
                
                if record.extract_match:
                    print(f"\nMatch Found: {record.extract_match}")
                    print(f"Match Probability: {record.match_probability}")
                    
                if record.utterance_results:
                    print(f"\nUtterance Analysis:")
                    for j, utterance in enumerate(record.utterance_results):
                        print(f"  Utterance {j+1} (Index {utterance['utterance_index']}):")
                        print(f"    Match Probability: {utterance['match_probability']}")
                        labeled_text = ""
                        for part in utterance['utterance_parts']:
                            if part['label']:
                                labeled_text += f"[{part['label']}: {part['text']}]"
                            else:
                                labeled_text += part['text']
                        print(f"    Text: {labeled_text}")
                
            elif record.operator_type == 'conversation-classify':
                print(f"\nClassification Result:")
                print("-" * 30)
                print(f"Predicted Label: {record.predicted_label}")
                print(f"Predicted Probability: {record.predicted_probability}")
                
                if record.label_probabilities:
                    print(f"All Label Probabilities:")
                    for label, prob in record.label_probabilities.items():
                        print(f"  {label}: {prob}")
            
            print(f"\nTranscript SID: {record.transcript_sid}")
            print(f"URL: {record.url}")
            print("=" * 60)
            print()
            
    except Exception as e:
        print(f"Error fetching operator results for transcript {transcript_record.sid}: {e}")
    
    print("=" * 80)
    print()

Getting operator results for 1 transcripts...

TRANSCRIPT: GT17c5c6b21b6e45d88549a5b148283baf
Found 11 operator results:

OPERATOR RESULT #1
Name: Virtual Agent Performance
Type: text-generation
Operator SID: LY69843e95cea04823b7d80cd08942978a

Text Generation Result:
------------------------------
1) The virtual agent was somewhat consultative, as it attempted to clarify the customer's needs when there was confusion about the questions being asked. For instance, when the customer mentioned "a trilha é melhor que a seus face," the agent asked for clarification, indicating a willingness to understand the customer's request better. However, the follow-up questions were limited, and the agent did not delve deeper into the customer's specific needs or preferences regarding Twilio and Salesforce, which could have led to a more tailored response.

2) The virtual agent was able to provide relevant information about Twilio and its comparison with Salesforce, but it struggled to fully grasp the

### Estratégia de Orquestração Omnicanal